In [28]:
import torch
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

def data_process(raw_text_iter):
  data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
  return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [29]:
train_data

tensor([   9, 3849, 3869,  ..., 2442, 4810,    3])

* narrow 역할
    * 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들) 은 잘라냅니다.

In [32]:
# bsz = 11
# nbatch = train_data.size(0) // bsz
# print(train_data.shape)
# train_data = train_data.narrow(0, 0, nbatch * bsz)
# print(train_data.shape)
# train_data.view(bsz,-1).t().contiguous()

In [33]:
def batchify(data, bsz):
    # 데이터셋을 bsz 파트들로 나눕니다.
    nbatch = data.size(0) // bsz
    # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들) 은 잘라냅니다.
    data = data.narrow(0, 0, nbatch * bsz)
    # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

In [34]:
train_data

tensor([[    9,    59,   564,  ..., 11652,  2435,     1],
        [ 3849,    12,   300,  ...,    47,    30,  1990],
        [ 3869,   315,    19,  ...,    97,  7720,     4],
        ...,
        [  587,  4011,    59,  ...,     1,  1439, 12313],
        [ 4987,    29,     4,  ...,  3165, 17106,  2060],
        [    6,     8,     1,  ...,    62,    18,     2]])

* get_batch() 
    * 함수는 트랜스포머 모델을 위한 입력과 타겟 시퀀스를 생성합니다. 
    * 이 함수는 소스 데이터를 bptt 길이를 가진 덩어리로 세분화 합니다. 
    * 언어 모델링 과제를 위해서, 모델은 다음 단어인 Target 이 필요 합니다. 
    * 예를 들어, bptt 의 값이 2 라면, 우리는 i = 0 일 때 다음의 2 개의 변수(Variable) 를 얻을 수 있습니다


* BPTT
    * BPTT를 통해 RNN의 가중치 행렬의 미분을 계산해보면 아래와 같이 최종적으로 미분의 곱으로 이루어진 항이 계산된다
    * 시퀀스 길이가 길어지는 경우 BPTT가 불안정해지므로 길이를 끊는 것이 필요하다. 이 방법을 Truncated BPTT라고 부른다.
    

In [103]:
bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len] # .reshape(-1)
    return data, target

bptt_list  = [i for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt))]
train_data[0:0+1] , train_data[0+1:0+1+1]
#get_batch(train_data , bptt_list[0])

(tensor([[    9,    59,   564,   223,   443, 13627,     2,   539,  2872,  2464,
              0,   313,  4513,     1,     5,    47,    66, 11652,  2435,     1]]),
 tensor([[ 3849,    12,   300,  6302,  3989,  1930, 10559,   451,     4,     7,
              2,  1511, 10115,   942,  2439,   572,     1,    47,    30,  1990]]))

In [104]:
train_data

tensor([[    9,    59,   564,  ..., 11652,  2435,     1],
        [ 3849,    12,   300,  ...,    47,    30,  1990],
        [ 3869,   315,    19,  ...,    97,  7720,     4],
        ...,
        [  587,  4011,    59,  ...,     1,  1439, 12313],
        [ 4987,    29,     4,  ...,  3165, 17106,  2060],
        [    6,     8,     1,  ...,    62,    18,     2]])

In [105]:
ntokens = len(vocab) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value

In [106]:
from torch import nn
ninp = emsize
encoder = nn.Embedding(ntokens, ninp)


In [107]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [108]:
src_mask = generate_square_subsequent_mask(bptt)

In [110]:
import math 

In [111]:
class PositionalEncoding(nn.Module):
    
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


In [112]:
from transformer_basic import TransformerEncoderLayer , TransformerDecoderLayer
from torch import Tensor
class TransformerEncoder(nn.Module):
    def __init__(
        self, 
        num_layers: int = 6,
        dim_model: int = 512, 
        num_heads: int = 8, 
        dim_feedforward: int = 2048, 
        dropout: float = 0.1, 
    ):
        super().__init__()
        self.layers = nn.ModuleList([
            TransformerEncoderLayer(dim_model, num_heads, dim_feedforward, dropout)
            for _ in range(num_layers)
        ])

    def forward(self, src: Tensor) -> Tensor:
        for layer in self.layers:
            src = layer(src)
        return src

class TransformerDecoder(nn.Module):
    def __init__(
        self, 
        num_layers: int = 6,
        dim_model: int = 512, 
        num_heads: int = 8, 
        dim_feedforward: int = 2048, 
        dropout: float = 0.1, 
    ):
        super().__init__()
        self.layers = nn.ModuleList([
            TransformerDecoderLayer(dim_model, num_heads, dim_feedforward, dropout)
            for _ in range(num_layers)
        ])
        self.linear = nn.Linear(dim_model, dim_model)

    def forward(self, tgt: Tensor, memory: Tensor) -> Tensor:
        for layer in self.layers:
            tgt = layer(tgt, memory)
        return torch.softmax(self.linear(tgt), dim=-1)

In [113]:
pos_encoder = PositionalEncoding(ninp, dropout)
transformer_encoder  = TransformerEncoder(num_layers=6,dim_model=ninp,dropout=dropout)
transformer_decoder  = TransformerDecoder(num_layers=6,dim_model=ninp,dropout=dropout)

In [119]:
data, targets = get_batch(train_data, bptt_list[0]) ## question
print(data.shape,targets.shape)
src = encoder(data) * math.sqrt(ninp) ## question
print(src.shape)
src = pos_encoder(src)
enc_output = transformer_encoder(src)

torch.Size([35, 20]) torch.Size([35, 20])
torch.Size([35, 20, 200])


In [116]:
tgt = encoder(targets) * math.sqrt(ninp) ## question
tgt = pos_encoder(tgt)

In [118]:
transformer_decoder(tgt , enc_output)

tensor([[[0.0061, 0.0039, 0.0072,  ..., 0.0050, 0.0021, 0.0060],
         [0.0065, 0.0061, 0.0082,  ..., 0.0020, 0.0039, 0.0012],
         [0.0028, 0.0049, 0.0025,  ..., 0.0143, 0.0021, 0.0044],
         ...,
         [0.0051, 0.0043, 0.0042,  ..., 0.0053, 0.0028, 0.0036],
         [0.0023, 0.0083, 0.0037,  ..., 0.0086, 0.0025, 0.0014],
         [0.0019, 0.0047, 0.0041,  ..., 0.0059, 0.0074, 0.0022]],

        [[0.0057, 0.0027, 0.0045,  ..., 0.0047, 0.0110, 0.0028],
         [0.0032, 0.0016, 0.0107,  ..., 0.0034, 0.0029, 0.0036],
         [0.0015, 0.0042, 0.0060,  ..., 0.0072, 0.0036, 0.0021],
         ...,
         [0.0029, 0.0038, 0.0029,  ..., 0.0014, 0.0099, 0.0033],
         [0.0031, 0.0039, 0.0033,  ..., 0.0177, 0.0068, 0.0033],
         [0.0032, 0.0029, 0.0046,  ..., 0.0051, 0.0059, 0.0011]],

        [[0.0017, 0.0026, 0.0067,  ..., 0.0024, 0.0031, 0.0100],
         [0.0036, 0.0050, 0.0111,  ..., 0.0014, 0.0046, 0.0056],
         [0.0013, 0.0028, 0.0022,  ..., 0.0048, 0.0046, 0.

## 질문 (21/06/21)
* 왜 ninp를 임베딩 후에 곱해주는 지?
    * https://arxiv.org/pdf/1706.03762.pdf 참고 했다고 함. 
    * [reference]](https://discuss.pytorch.org/t/pytorch-transformers/76993)
    * scaling factor 같은 역할 논문 4pg 참고 
    
* 실제 코드에서는 attention에 src_mask라는 개념이 있음. 역할이 무엇인지 궁금함
    * [stackoverflow](https://stackoverflow.com/questions/62170439/difference-between-src-mask-and-src-key-padding-mask)
    * mask 2가지 종류 있음.
        * attn_mask – 2D or 3D mask that prevents attention to certain positions.
        * key_padding_mask – if provided, specified padding elements in the key will be ignored by the attention.
    * [medium](https://medium.com/analytics-vidhya/masking-in-transformers-self-attention-mechanism-bad3c9ec235c)
        * Masking is needed to prevent the attention mechanism of a transformer from “cheating” in the decoder when training 
    * 요약
        * attn_mask and key_padding_mask is used in Encoder's MultiheadAttention and Decoder's Masked MultiheadAttention.
        * memory_mask  is used in Decoder's MultiheadAttention mechanism as pointed out here.
        
    * 예제
        * I Love it이라고 할 때, 우리가 원하는 것은 I라는 것이 나올 때 Love를 것을 예측하고 싶은 것이고, I Love가 나올 때 It이라는 것을 예측하고 싶은 것임
        * 이전 토큰을 모두 사용하여 예측할 때주의 메커니즘이 다음 위치에서 토큰에 관한 정보를 공유하는 것을 원하지 않기 위해서 mask를 사용함

* PositionalEncoding을 저렇게 구현하는 이유 궁금
    * 해당 글에서도 나와 같은 의문을 가지고 있어서 정리한 글[참고]](https://kazemnejad.com/blog/transformer_architecture_positional_encoding/)
    * 언어에서 위체에 대한 정보는 중요한데, 이것은 transformer architecture가 되면서 이러한 정보를 가질 수가 없게 되었다. 
        * 기존 구조만으로는 위치/순서가 의미가 없어짐 
        * 단어의 순서를 표현할 수 있느 것이 필요함.
    * 그래서 그것의 위치에 대한 정보를 각 단어에 추가하는 것이다. 우리는 이것을 위치 인코딩인 "piece of information"라고 부른다.
    * 초기 아이디어 1
        * range를 주고 숫자 정보를 줌 [0~1]
            * 소개할 문제 중 하나는 특정 범위 내에서 얼마나 많은 단어가 존재하는지 알 수 없다는 것입니다. 다시 말해서, 시간 단계 델타는 서로 다른 문장들에 걸쳐 일관된 의미를 가지고 있지 않다.
    * 초기 아이디어 2 
        * 각각의 타입 스탭에 선형적으로 수자를 할당하는 방법 
            * 첫번째 단어 1 / 두번쨰 단어 2 
        * 값이 커질뿐만 아니라 학습된 것보다 더 긴 문장에서 문제가 발생함.(일반화 어려움)
    * 그래서 다음을 만족해야 함
        * It should output a unique encoding for each time-step (word’s position in a sentence)
        * Distance between any two time-steps should be consistent across sentences with different lengths.
        * Our model should generalize to longer sentences without any efforts. Its values should be bounded.
        * It must be deterministic.

* BPTT를 하는 이유는 알았는데, 꼭 필요한 사항인건지 궁금